In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

import folium
from folium.plugins import HeatMap

import random

ModuleNotFoundError: No module named 'pandas'

In [ ]:
austin = pd.read_csv("listings.csv", index_col=0)
df = pd.DataFrame(austin)
random.seed(123)

Mounted at /content/drive
/content/drive/MyDrive/STAT315_FinalProject


**Project Introduction and Data Source**

Our study is motivated by the need to analyze Airbnb pricing in Austin, TX and gain a better understanding of what factors impact these rates. To do this, we used Airbnb.com data collected from October to November 2021. This data was scraped from the website and recorded. In total, the dataset had 16 columns with data. This included factors such as price, neighborhood, room type, minimum nights of stay available, total number of reviews, number of host listings, yearly availability, and number of reviews left by users in the last 12 months.

To protect location anonymity, Airbnb anonymizes location information for listings. Because of this, the location for a listing in the data will be 0-450 feet from the actual address. Furthermore, listings in the same building are anonymized individually, so they may appear scattered around the actual address.

For our analysis, we decided to explore how different variables, such as location factors, affect our target variable price. We hope that an understanding of these variables and their trends can help users book more affordable rentals and facilitate reasonable rental pricing by hosts.

In [ ]:
# Drop any rows with missing latitude, longitude, or price
df = df.dropna(subset=['latitude', 'longitude', 'price'])

df = df.drop(columns=['license', 'host_name', 'name', 'host_id', 'id'], errors='ignore')

df['price'] = df['price'].fillna(df['price'].median())
df['reviews_per_month'] = df['reviews_per_month'].fillna(df['reviews_per_month'].median())

df['log_minimum_nights'] = np.log1p(df['minimum_nights'])
df['log_number_of_reviews'] = np.log1p(df['number_of_reviews'])
df['log_price'] = np.log1p(df['price'])


**Data Cleaning and Preparation**

After exploring the data, we realized that we had missing values in the neigh-
borhood groups, reviews per month, and price columns.
Since the neighborhood groups column actually had no data points, we ended
up dropping the entire column. We filled in the missing values with 0 for
the reviews per month column, assuming that the missing data points had no
reviews. For the price column, we imputed the missing data using the median
price value.
a
We discovered that there was high right skew in the minimum nights, number of
reviews, and price columns. As a result, we decided to do a log transformation
on all three. Since there were outliers on price, our response variable, we removed listings
that were over $1000 per night.

After seeing the importance of lattitude and longitude to our MLP we exper-
imented with creating new features from them that would help with interpre-
tation and hopefully would increase the performance of our model. Our first
attempt was to simply find the coordinates for downtown Austin and then mea-
sure listing’s distance from there and record them in the column "distance to
center".

***Exploratory Data Analysis***

Before we delve into complex modeling, we felt that it would be beneficial to gain a deeper understanding of our target variable. To do this, we begain with some exploratory data analysis focusing on the distriibution of price.

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

plt.figure(figsize=(10,6))
sns.histplot(df['price'].dropna(), bins=50, kde=True) # Plots the distribution of price

plt.title('Distribution of Listing Prices', fontsize=18)
plt.xlabel('Price ($)', fontsize=14)
plt.ylabel('Number of Listings', fontsize=14)

plt.show()

ModuleNotFoundError: No module named 'matplotlib'

From this original plot, we observed that the distribution of price is very left skewed. This could indicate the presence of high outliers. To better account for this, we tried a similiar plot with the natural log of price.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

plt.figure(figsize=(10,6))
sns.histplot(df['price'].dropna(), bins=50, kde=True, log_scale=(True, False)) #Puts the x-axis in log scale so the distribution isn't so clustered

plt.title('Distribution of Listing Prices', fontsize=18)
plt.xlabel('Price ($)', fontsize=14)
plt.ylabel('Number of Listings', fontsize=14)

plt.show()

Taking the natural log of price provided a drastically improved visual to see the shape of listing prices. This transformed distribution revealed a more balanced spread of listing prices, highlighting a clear central tendency and reducing the visual dominance of high-end outliers.

In [ ]:
sns.set(style="whitegrid")

df_filtered = df[df['price'] > 0] # Only keeps listings with positive prices

plt.figure(figsize=(12,8))
sns.histplot(
    data=df_filtered,
    x='price',
    hue='room_type', # color by room type
    bins=50,
    kde=True,
    log_scale=(True, False)
)

plt.title('Distribution of Listing Prices by Room Type (Log Scale)', fontsize=18)
plt.xlabel('Price ($) - Log Scale', fontsize=14)
plt.ylabel('Number of Listings', fontsize=14)

plt.legend(title='Room Type', labels=df_filtered['room_type'].unique())
plt.show()


In [ ]:
sns.set(style="whitegrid")

top_zipcodes = df_filtered['neighbourhood'].value_counts().head(5).index.tolist()

df_zip = df_filtered[df_filtered['neighbourhood'].isin(top_zipcodes)].copy()
df_zip['neighbourhood'] = df_zip['neighbourhood'].astype(str)

plt.figure(figsize=(12,8))
sns.boxplot(
    data=df_zip,
    x='neighbourhood',
    y='price',        
    palette='Set2'   
)

plt.yscale('log')

plt.title('Listing Prices by ZIP Code (Top 5)', fontsize=18)
plt.xlabel('ZIP Code', fontsize=14)
plt.ylabel('Price ($) - Log Scale', fontsize=14)

plt.show()

Next, we wanted to visualize how price is distributed across the top five most expensive ZIP codes in the Austin, TX metropolitan area. To assist with visibility and account for outliers, we decided to again take the natural log of price.

This boxplot shows a lot of price variation within each of these ZIP codes. Comparitively, across these codes, ZIP 78701 has the highest median price. This ZIP code specifically also has the largest spread of prices, while ZIP 78745 has the smallest spread. Each ZIP code group still has a large spread, likely due to the large amount of outliers.

In [ ]:
sns.set(style="whitegrid")

top_zipcodes = df_filtered['neighbourhood'].value_counts().head(5).index.tolist()

df_zip = df_filtered[df_filtered['neighbourhood'].isin(top_zipcodes)].copy()

df_zip['neighbourhood'] = df_zip['neighbourhood'].astype(str)

plt.figure(figsize=(12,8))
sns.histplot(
    data=df_zip,
    x='price',
    hue='neighbourhood',
    bins=50,
    kde=True,
    log_scale=(True, False)
)

plt.title('Distribution of Listing Prices by ZIP Code (Top 5)', fontsize=18)
plt.xlabel('Price ($) - Log Scale', fontsize=14)
plt.ylabel('Number of Listings', fontsize=14)

plt.legend(title='ZIP Code')
plt.show()

Similarly to our previous plot, we utilized a box plot to visualize the distribution of pricing, this time across room types. From this, we can see that entire rooms/apartments are expensive on average but have a high variation between prices. Private rooms are the second cheapest on average, but also have a high price variation. However, shared rooms are on average the cheapest and are relatively consistent in price, especially compared to other room types. In contrast, hotel rooms are on average the most expensive, but have the most consistent price rates. Most importantly, it is important to observe how the majority of our price variation occurs in entire homes/apartments and private rooms. These groups simultaneously contain some of our most expensive and cheapest options, indicating that room type alone is probably not the only factor contributing to price.

In [ ]:
# Step 4: Make a HeatMap data list
heat_data = [[row['latitude'], row['longitude'], row['price']] for index, row in df.iterrows()]

# Step 5: Create a base map centered around Austin
m = folium.Map(location=[30.2672, -97.7431], zoom_start=12)

# Step 6: Add the HeatMap
HeatMap(heat_data,
        radius=15,
        blur=10,
        max_zoom=1).add_to(m)
# Step 7: Add Tourist Attractions
tourist_attractions = [
    {"name": "Texas State Capitol", "lat": 30.2747, "lon": -97.7404},
    {"name": "Lady Bird Lake", "lat": 30.2604, "lon": -97.7490},
    {"name": "Zilker Park", "lat": 30.2669, "lon": -97.7725},
    {"name": "Barton Springs Pool", "lat": 30.2644, "lon": -97.7720},
    {"name": "South Congress Avenue", "lat": 30.2491, "lon": -97.7496},
    {"name": "The University of Texas at Austin", "lat": 30.2850, "lon": -97.7345},
    {"name": "Sixth Street Historic District", "lat": 30.2676, "lon": -97.7393},
    {"name": "Mount Bonnell", "lat": 30.3215, "lon": -97.7737},
    {"name": "Blanton Museum of Art", "lat": 30.2803, "lon": -97.7370}
]

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'cadetblue', 'pink']

# Add markers for each tourist attraction with a different color
for idx, place in enumerate(tourist_attractions):
    folium.Marker(
        location=[place["lat"], place["lon"]],
        popup=place["name"],
        icon=folium.Icon(color=colors[idx % len(colors)], icon='info-sign')
    ).add_to(m)

# Show the map
m

For our last exploratory analysis, we wanted to explore the pricing distribution on a geo-spatial level. First, we noticed that most of our more expensive lisitngs were centralized around the "downtown" Austin area, with cheaper rentals dispersed across less "touristy" areas. From this heat map, we also observed some gapping throughout unexpected areas. This left us wondering how proximity to recreational attractions may affect price and rental locations. So, we were able to obtain coordinates of popular attractions in Austin and overlay them on our map. This did potentially explain some of our observed trends, including the gapping, as some attractions are on private land that typically do not host Airbnbs. For example, the University of Texas and the Texas State Capitol are not available for Airbnbs due to the presence of student housing and ownership by the state. It makes sense that many renters are willing to pay more to be as close as possible to the city's attractions, so Airbnb hosts listing as close as possible to them are seemingly pricing these properties higher on average than those that are further away (for example, Mount Bonnell).

**Data Analysis**

In [ ]:

dfmlp = pd.DataFrame(austin)

#drop useless columns
dfmlp = dfmlp.drop(columns=['license', 'host_name', 'name', 'host_id', 'id'], errors='ignore')

#created clusters of close together landmarks 
clusters = {
    "downtown_cluster": [
        (30.2747, -97.7404),  # Texas State Capitol
        (30.2604, -97.7490),  # Lady Bird Lake
        (30.2676, -97.7393),  # Sixth Street Historic District
        (30.2850, -97.7345),  # University of Texas at Austin
        (30.2803, -97.7370)   # Blanton Museum of Art
    ],
    "zilker_cluster": [
        (30.2669, -97.7725),  # Zilker Park
        (30.2644, -97.7720)   # Barton Springs Pool
    ],
    "northwest_cluster": [
        (30.3215, -97.7737)   # Mount Bonnell
    ],
    "south_congress_cluster": [
        (30.2491, -97.7496)   # South Congress Avenue
    ]
}

# transformed last review day to days since last review
dfmlp['last_review'] = pd.to_datetime(dfmlp['last_review'], errors='coerce')
today = pd.to_datetime('today')
dfmlp['days_since_last_review'] = (today - dfmlp['last_review']).dt.days
dfmlp['days_since_last_review'] = dfmlp['days_since_last_review'].fillna(dfmlp['days_since_last_review'].max())


dfmlp = dfmlp.drop(columns=['last_review'])

# filled reviews_per_month na values with 0
dfmlp['reviews_per_month'] = dfmlp['reviews_per_month'].fillna(0)

# function to calculate distance between lattitude
def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((69 * (lat1 - lat2))**2 + (54.6 * (lon1 - lon2))**2)

# Create distance to each cluster
for cluster_name, locations in clusters.items():
    distances = [calculate_distance(dfmlp['latitude'], dfmlp['longitude'], lat, lon) for lat, lon in locations]
    # Take the minimum distance to any point in the cluster
    dfmlp[f"dist_to_{cluster_name}"] = np.min(distances, axis=0)

# transform skewed factors
dfmlp['log_minimum_nights'] = np.log1p(df['minimum_nights'])
dfmlp['log_number_of_reviews'] = np.log1p(df['number_of_reviews'])

# list factors by type
categorical = ['neighbourhood', 'neighbourhood_group', 'room_type']
numerical = [
    'log_minimum_nights', 'log_number_of_reviews', 'calculated_host_listings_count',
    'availability_365', 'number_of_reviews_ltm', 'reviews_per_month',
    'latitude', 'longitude', 'days_since_last_review',
    'dist_to_downtown_cluster', 'dist_to_zilker_cluster', 'dist_to_northwest_cluster', 'dist_to_south_congress_cluster'
]

# removed outliers
price_cap = 1000  
dfmlp = dfmlp[dfmlp['price'] <= price_cap]

# drop rows with missing price data 
df_cleanmlp = dfmlp[categorical + numerical + ['price']].dropna(subset=['price'])

# log transform price because it is still skewed 
X = df_cleanmlp[categorical + numerical]
y = np.log1p(df_cleanmlp['price'])  

#hot one encoding my categorical variables
temp_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
temp_encoder.fit(X[categorical])


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),  # numeric features: impute + scale
        ('cat', OneHotEncoder(categories=temp_encoder.categories_, handle_unknown='ignore', sparse_output=False), categorical)  # categorical: one-hot encode
    ]
)

pipeline_for_tuning = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(max_iter=10000, random_state=42))
])

# grid of parameters for hyperparameter tuning
param_grid = {
    'regressor__hidden_layer_sizes': [(50,), (100,), (100, 50), (150, 100)],
    'regressor__alpha': [0.0001, 0.001, 0.01],
    'regressor__learning_rate_init': [0.001, 0.01, 0.1]
}

#grid search to run all combinations of hyperparameters
search = GridSearchCV(
    pipeline_for_tuning,
    param_grid,
    cv=3,
    scoring='r2',
    verbose=2,
    n_jobs=-1
)

search.fit(X, y)

print("\nBest parameters from tuning:")
print(search.best_params_)

# regression model
best_mlp = MLPRegressor(
    hidden_layer_sizes=search.best_params_['regressor__hidden_layer_sizes'],
    alpha=search.best_params_['regressor__alpha'],
    learning_rate_init=search.best_params_['regressor__learning_rate_init'],
    max_iter=10000,
    random_state=42
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', best_mlp)
])

# kfolds resampling method
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# holders for performance metrics
importances_list = []
mae_list = []
rmse_list = []
r2_list = []

feature_names = None 

# run kfolds 
for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    pipeline.fit(X_train, y_train)

   
    y_pred = pipeline.predict(X_test)
    y_pred = np.expm1(y_pred)
    y_test = np.expm1(y_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    r2_list.append(r2)

    
    X_test_transformed = pipeline.named_steps['preprocessor'].transform(X_test)

    
    result = permutation_importance(
        pipeline.named_steps['regressor'],
        X_test_transformed,
        y_test,
        n_repeats=10,
        random_state=22,
        n_jobs=-1
    )
    importances_list.append(result.importances_mean)

    
    if feature_names is None:
        cat_features = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical)
        num_features = numerical
        feature_names = list(num_features) + list(cat_features)

    print(f"Fold {fold} completed. MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.3f}")


mean_importances = np.mean(importances_list, axis=0)

assert len(feature_names) == len(mean_importances), f"Mismatch: {len(feature_names)} names vs {len(mean_importances)} importances."


importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': mean_importances
}).sort_values(by='Importance', ascending=False)


print("\nAverage Model Performance Across Folds:")
print(f"Average MAE: {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
print(f"Average RMSE: {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
print(f"Average R²: {np.mean(r2_list):.3f} ± {np.std(r2_list):.3f}")


metrics_df = pd.DataFrame({
    'Fold': list(range(1, 6)),
    'MAE': mae_list,
    'RMSE': rmse_list,
    'R2': r2_list
})

In order to get a better understanding about how our factors impacted price we trained two models, linear regression and multilayer perceptron (MLP). We used a k-folds cross validation approach to training both models in order to get a stable estimate of their respecitve performances. In order to have a fair comparison we also made sure to train both models on identical feature sets. We used GridSearchCV in order to test a wide array of hyperparameters for the MLP models as well as preparing the factors, but both the MLP and the linear regression failed to get RMSE's of less than 600. Running a variable importance plot we found that latitude and longitude were the most important features especially for the MLP. We attempted to make this location information more interpretable and increase model performance by finding the coordinates for downtown Austin as defined by Apple Maps and finding the distance from each listing and encoding it as 'distance_to_center'. This led to a dramatic improvement for our models in terms of RMSE. After this success we decided to find more landmarks in Austin area and encode the distance of listings from them in their own factors. Since many landmarks are close together it made sense to create clusters of landmarks and to only record the minimum distance from each listing to one of the locations in that cluster. This, however, only led to a moderate improvement in model performance. The MLP ended up with a RMSE of 159.98. The standard deviation for prices in general were 195.71 which means our model is superior to estimating all airbnb prices with the mean but only somewhat.

In [ ]:
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': mean_importances
}).sort_values(by='Importance', ascending=False)

def classify_feature(feature_name):
    if feature_name.startswith('neighbourhood_'):
        return 'Neighbourhood'
    elif feature_name.startswith('room_type_'):
        return 'Room Type'
    else:
        return feature_name

importance_df['Feature_Type'] = importance_df['Feature'].apply(classify_feature)

grouped_importance = importance_df.groupby('Feature_Type')['Importance'].sum().reset_index()

plt.figure(figsize=(8,6))
sns.barplot(data=grouped_importance, x='Importance', y='Feature_Type', palette='viridis')
plt.title('Total Feature Importance by Group (5-Fold CV)', fontsize=16)
plt.xlabel('Total Permutation Importance', fontsize=14)
plt.ylabel('Feature Group', fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


print(grouped_importance)

Looking at our final variable importance plot from our MLP model we see that the most important features almost all relate to distance. This was expected as location is commonly seen as the most important aspect about real estate. What was suprising however, was how distance to South Congress cluster was more important than distance to downtown cluster. We had expected the distance to downtown cluster to be the most important. Somewhat unsurprisingly, distance to north west cluster was considered of little importance to predicting price. This suggests that proximity to Mount Bonnell is unimportant to most consumers in determining what they are willing to pay for an Airbnb. The most important non-location factor was reviews_per_month. This also makes sense, as having a lot of reviews suggests that consumers on average are more willing to trust the hosts and the listing is more of a hot spot.

While we extracted some insights especially about the importance of location on price, our model was only somewhat superior to using a fixed estimator such as the mean in terms of predicting price. This indicates that our features were insufficient to providing an accurate estimate of the price for an Airbnb. Future study should include factors such as photo quality, crime rates, or even seasonal popularity in order to get a more complete picture.

Furthermore, it is worth noting that our importance values are all very low. This suggests that the features may be highly correlated with each other, that our data is noisy, or that there is simply a weak relationship between the features and our response variable. If we had more time, we would explore multicolinearity further using tools such as a correlation matrix or a variance inflation factor and remove highly correlated features.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


lr_mae_list = []
lr_rmse_list = []
lr_r2_list = []

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    linear_pipeline.fit(X_train, y_train)
    y_pred = linear_pipeline.predict(X_test)

    # Revert log transform
    y_pred_original = np.expm1(y_pred)
    y_test_original = np.expm1(y_test)

    # Metrics
    mae = mean_absolute_error(y_test_original, y_pred_original)
    rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
    r2 = r2_score(y_test_original, y_pred_original)

    lr_mae_list.append(mae)
    lr_rmse_list.append(rmse)
    lr_r2_list.append(r2)

    print(f"Linear Model - Fold {fold}: MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.3f}")

print("\n=== Linear Regression Model Performance Across Folds ===")
print(f"Average MAE: {np.mean(lr_mae_list):.2f} ± {np.std(lr_mae_list):.2f}")
print(f"Average RMSE: {np.mean(lr_rmse_list):.2f} ± {np.std(lr_rmse_list):.2f}")
print(f"Average R²: {np.mean(lr_r2_list):.3f} ± {np.std(lr_r2_list):.3f}")

The RMSE of our linear regression model was 171.28. This means that on average, we are about \$171.27 away from predicting the right price. While it is inferior in terms of our MLP model, we expected it to perform much worse. Initially before our feature engineering, the RMSE of the linear regression model was much worse than for our MLP model. This suggests that in the course of creating new and simpler features from more complicated ones such as longitude and latitude,  we essentially are able to see a more linear relationship between our factors and the response. This is one of the limitations of the linear regression model: the data may not be linear. In addition, our estimates may be unstable since some of the features could be highly correlated. The biggest limitation is that this model does not show us which features are significant. Thus, we need to do further analysis.

**Interpretation**

From our feature importance plot, we see that the most important features are Room Type, Distance to the Downtown, South Congress, and Zilker Clusters, and neighborhood (zip code). Thus, we suggest that travelers who are wanting to save money should book listings that are farther from the main hotspots of Austin: Downtown, South Congress, and Zilker Park. Furthermore, highly affluent neighborhoods may also be more expensive than those that are poorer. Finally, travelers who book for listings that are not shared can expect to spend more money.

**Contribution Report**

Abby contributed to defining the question, the exploratory data analysis, and
building the MLP model. Zach contributed to the exploratory data analysis and interpretation.
Michael contributed to cleaning the data and building the MLP model. Joyce
contributed to cleaning the data and building the linear regression model.
We all contributed to the results and conclusion.